# V9

In [1]:
import pandas as pd
import pypsa
import os
import numpy as np

from src.utils_multiperiod import *
from src.scenario_utils import *
from src.config import *

n = pypsa.Network()
base_path = os.getcwd()

In [2]:
# Set modeling timeline
start_year = 2025       # Start from when existing generator began
end_year = 2050        # End of modeling period
investment_years = list(range(start_year, end_year))

r = 0.08 # discount rate / WACC

ISP Costs

In [3]:
inputs_path = os.path.join(base_path, 'data', 'INPUTS.xlsx')
tech_params = get_isp_tech_params(inputs_path)
fuel_costs = get_isp_fuel_costs(inputs_path)
build_costs = get_isp_tech_build_costs(inputs_path, sheet_name='TECH_BUILD_COSTS')

GENCOST

In [4]:
capex_path = os.path.join(base_path, "data", "GENCOST_CAPEX.csv")
cost_capex = get_gencost_capex(capex_path)

cost_capex.head(5)

,YEAR,BESS_12HR_CAPEX,BESS_1HR_CAPEX,BESS_24HR_CAPEX,BESS_2HR_CAPEX,BESS_4HR_CAPEX,BESS_8HR_CAPEX,CCGT_CAPEX,GAS_RECIP_CAPEX,GAS_CCS_CAPEX,H2_RECIP_CAPEX,OCGT_LRG_CAPEX,OCGT_SML_CAPEX,SOLAR_PV_CAPEX,WIND_CAPEX
0,2024,3816000.0,910000.0,7008000.0,1216000.0,1692000.0,2752000.0,2455000.0,1980000.0,5802000.0,2071000.0,1310000.0,2426000.0,1463000.0,3223000.0
1,2025,3720000.0,889000.0,6840000.0,1186000.0,1652000.0,2688000.0,2300000.0,1982000.0,5404000.0,2073000.0,1311000.0,2428000.0,1414000.0,3113000.0
2,2026,3624000.0,864000.0,6672000.0,1154000.0,1608000.0,2616000.0,2144000.0,1984000.0,5025000.0,2075000.0,1312000.0,2431000.0,1369000.0,3006000.0
3,2027,3564000.0,848000.0,6552000.0,1134000.0,1580000.0,2568000.0,2034000.0,1980000.0,4765000.0,2071000.0,1310000.0,2426000.0,1322000.0,2900000.0
4,2028,3492000.0,833000.0,6432000.0,1114000.0,1552000.0,2520000.0,1977000.0,1976000.0,4634000.0,2067000.0,1307000.0,2421000.0,1284000.0,2797000.0


In [5]:
variable_cost_path = os.path.join(base_path, "data", "GENCOST_VARIABLE.csv")
cost_variable = get_gencost_variables(variable_cost_path, end_year=2055)
cost_variable.head(5)

,YEAR,CCGT_BUILDTIME,CCGT_EFF,CCGT_FOM,CCGT_LIFESPAN,CCGT_VOM,GAS_CCS_BUILDTIME,GAS_CCS_CCS,GAS_CCS_EFF,GAS_CCS_FOM,...,OCGT_LARGE_FUEL,H2_RECIP_FUEL,GAS_CCS_FUEL,GAS_RECIP_FUEL,CCGT_FUEL,OCGT_SMALL_MC,OCGT_LARGE_MC,GAS_CCS_MC,GAS_RECIP_MC,CCGT_MC
0,2025,1.5,0.51,15000.0,25.0,4.1,1.5,1.9,0.44,22500.0,...,59.94,151.38,59.94,59.94,59.94,182.6,189.736364,144.227273,154.695122,121.629412
1,2026,1.5,0.51,15000.0,25.0,4.1,1.5,1.9,0.44,22500.0,...,59.94,151.38,59.94,59.94,59.94,182.6,189.736364,144.227273,154.695122,121.629412
2,2027,1.5,0.51,15000.0,25.0,4.1,1.5,1.9,0.44,22500.0,...,59.94,151.38,59.94,59.94,59.94,182.6,189.736364,144.227273,154.695122,121.629412
3,2028,1.5,0.51,15000.0,25.0,4.1,1.5,1.9,0.44,22500.0,...,59.94,151.38,59.94,59.94,59.94,182.6,189.736364,144.227273,154.695122,121.629412
4,2029,1.5,0.51,15000.0,25.0,4.1,1.5,1.9,0.44,22500.0,...,59.94,151.38,59.94,59.94,59.94,182.6,189.736364,144.227273,154.695122,121.629412


In [6]:
inputs_df = pd.merge(cost_capex, cost_variable, on="YEAR", how="outer").bfill()
# Example usage - replace the original block with:
techs_to_keep = ["SOLAR_PV", "WIND", "BESS_1HR", "BESS_2HR", "BESS_4HR", "BESS_8HR", "BESS_12HR", "BESS_24HR", "OCGT_SML", "GAS_RECIP"]

inputs_df = filter_and_process_input_costs(
    inputs_df=inputs_df,
    techs_to_keep=techs_to_keep,
    annuitise_capex=True, 
    discount_rate=r,
    model_horizon= end_year - start_year + 1,  # Adjusted to match the modeling period
)

inputs_df.head(5)

No FOM column for BESS. Assuming FOM = 0.
No FOM column for BESS. Assuming FOM = 0.
No FOM column for BESS. Assuming FOM = 0.
No FOM column for BESS. Assuming FOM = 0.
No FOM column for BESS. Assuming FOM = 0.
No FOM column for BESS. Assuming FOM = 0.
No FOM column for OCGT_SML. Assuming FOM = 0.
No FOM column for SOLAR_PV. Assuming FOM = 0.


,BESS_12HR_CAPEX,BESS_1HR_CAPEX,BESS_24HR_CAPEX,BESS_2HR_CAPEX,BESS_4HR_CAPEX,BESS_8HR_CAPEX,GAS_RECIP_CAPEX,OCGT_SML_CAPEX,SOLAR_PV_CAPEX,WIND_CAPEX,...,GAS_RECIP_EFF,GAS_RECIP_FOM,GAS_RECIP_LIFESPAN,GAS_RECIP_VOM,WIND_BUILDTIME,WIND_EFF,WIND_FOM,WIND_LIFESPAN,GAS_RECIP_FUEL,GAS_RECIP_MC
YEAR,,,,,,,,,,,,,,,,,,,,,
2024,353007.195464,84181.485291,648289.943870,112488.666060,156522.058366,254579.612661,212564.110854,224422.289359,135337.926353,326150.469334,...,0.41,29400.0,25.0,8.5,1.0,1.0,28000.0,25.0,59.94,154.695122
2025,344126.511301,82238.835631,632748.746586,109713.452259,152821.773298,248659.156553,212749.125107,224607.303613,130805.077145,315974.685398,...,0.41,29400.0,25.0,8.5,1.0,1.0,28000.0,25.0,59.94,154.695122
2026,335245.827138,79926.157463,617207.549301,106753.224205,148751.459724,241998.643431,212934.139361,224884.824993,126642.256444,306076.422842,...,0.41,29400.0,25.0,8.5,1.0,1.0,28000.0,25.0,59.94,154.695122
2027,329695.399537,78446.043436,606106.694098,104903.081671,146161.260176,237558.301350,212564.110854,224422.289359,122294.421489,296270.667412,...,0.41,29400.0,25.0,8.5,1.0,1.0,28000.0,25.0,59.94,154.695122
2028,323034.886415,77058.436536,595005.838895,103052.939137,143571.060629,233117.959268,212194.082347,223959.753726,118779.150675,286742.433363,...,0.41,29400.0,25.0,8.5,1.0,1.0,28000.0,25.0,59.94,154.695122


In [7]:
rez_id=REZIDS.N3
reference_year=2016

# Solar Params
solar_type='SAT'
solar_degradation=0.0035
solar_life=tech_life.get("SOLAR_PV", None)

# Wind Params
wind_type='WH'
wind_degradation=None
wind_life=tech_life.get("WIND", None)

In [8]:
solar_trace = solar_trace_construction(
    start_year=start_year, 
    end_year=end_year, 
    rez_ids=rez_id, 
    reference_year=reference_year, 
    solar_type=solar_type, 
    annual_degradation=solar_degradation, 
    lifetime=solar_life,
    build_year=investment_years[0],
    multi_reference_year=False,
    year_type='calendar',
    investment_periods=investment_years,
    freq='h' 
)
wind_trace = wind_trace_construction(
    start_year=start_year, 
    end_year=end_year, 
    rez_ids=rez_id, 
    reference_year=reference_year, 
    wind_type=wind_type, 
    annual_degradation=wind_degradation, 
    lifetime=wind_life,
    build_year=investment_years[0],
    multi_reference_year=False,
    year_type='calendar',
    investment_periods=investment_years,
    freq='h'
)

In [9]:
max_output_trace = pd.concat([
    solar_trace,
    wind_trace,
    ], 
    axis=1,
    join='outer'
)
max_output_trace

SOLAR_2025  WIND_2025
period timestep                                  
2025   2025-01-01 00:00:00    0.000000   0.610608
       2025-01-01 01:00:00    0.000000   0.590224
       2025-01-01 02:00:00    0.000000   0.575205
       2025-01-01 03:00:00    0.000000   0.595563
       2025-01-01 04:00:00    0.000000   0.546717
...                                ...        ...
2049   2049-12-31 19:00:00    0.023506   0.015005
       2049-12-31 20:00:00    0.000000   0.010998
       2049-12-31 21:00:00    0.000000   0.012907
       2049-12-31 22:00:00    0.000000   0.058051
       2049-12-31 23:00:00    0.000000   0.147977

[219000 rows x 2 columns]

In [10]:
homer_trace = pd.read_csv(r'C:\Users\Jiuqi Shang\OneDrive - UNSW\Thesis\Coding\Project Final\AUS-PyPSA-HRES\data\other_traces\HOMER_TRACE_25YRS.csv', index_col=0, parse_dates=True)
hh_index = create_multiindex_snapshots(
    start_date="2025-01-01 00:00:00",
    end_date="2049-12-31 23:59:59",
    freq='30min',
    investment_periods=investment_years
)
homer_trace.index = hh_index
homer_trace

SOLAR_TRACE  WIND_TRACE
period timestep                                    
2025   2025-01-01 00:00:00          0.0    0.178370
       2025-01-01 00:30:00          0.0    0.178370
       2025-01-01 01:00:00          0.0    0.204150
       2025-01-01 01:30:00          0.0    0.204150
       2025-01-01 02:00:00          0.0    0.145452
...                                 ...         ...
2049   2049-12-31 21:30:00          0.0    0.163426
       2049-12-31 22:00:00          0.0    0.000000
       2049-12-31 22:30:00          0.0    0.000000
       2049-12-31 23:00:00          0.0    0.000000
       2049-12-31 23:30:00          0.0    0.000000

[438000 rows x 2 columns]

In [ ]:
homer_trace_hourly = homer_trace.resample('h', level='period').mean()
hourly_index = create_multiindex_snapshots(
    start_date=homer_trace.index[0],
    end_date=homer_trace.index[-1],
    freq='h',
    investment_periods=investment_years
)
homer_trace_hourly.index = hourly_index

In [ ]:
plot_vre_gen_profiles(homer_trace, freq='W', title='HOMER VRE Generation Profiles')

In [11]:
n.snapshots = homer_trace.index
n.investment_periods = investment_years

In [12]:
n.snapshot_weightings.loc[:, :] = 0.5
n.snapshot_weightings

objective  stores  generators
period timestep                                          
2025   2025-01-01 00:00:00        0.5     0.5         0.5
       2025-01-01 00:30:00        0.5     0.5         0.5
       2025-01-01 01:00:00        0.5     0.5         0.5
       2025-01-01 01:30:00        0.5     0.5         0.5
       2025-01-01 02:00:00        0.5     0.5         0.5
...                               ...     ...         ...
2049   2049-12-31 21:30:00        0.5     0.5         0.5
       2049-12-31 22:00:00        0.5     0.5         0.5
       2049-12-31 22:30:00        0.5     0.5         0.5
       2049-12-31 23:00:00        0.5     0.5         0.5
       2049-12-31 23:30:00        0.5     0.5         0.5

[438000 rows x 3 columns]

In [13]:
investment_weightings = calculate_investment_period_weightings(
    end_year=end_year,
    investment_period_years=investment_years,
    discount_rate=r
)
n.investment_period_weightings = investment_weightings
n.investment_period_weightings

,objective,years
period,,
2025,1.000000,1
2026,0.925926,1
2027,0.857339,1
2028,0.793832,1
2029,0.735030,1
2030,0.680583,1
2031,0.630170,1
2032,0.583490,1
2033,0.540269,1


Add Bus

In [14]:
carriers = [
    "GAS",
    "WIND",
    "SOLAR_PV",
    "BESS",
    "AC"
]

n.add(
    "Carrier",
    carriers,
    color=[
        "indianred",
        "dodgerblue", 
        "gold",   
        "yellowgreen",
        "black"
        ],
    co2_emissions=[
        0.6, 
        0, 
        0, 
        0,
        0
    ],
)

Index(['GAS', 'WIND', 'SOLAR_PV', 'BESS', 'AC'], dtype='object')

In [15]:
n.add(
    "Bus", 
    "electricity",
    carrier="AC",
    )

Index(['electricity'], dtype='object')

In [16]:
load_fix = pd.Series(40, index=n.snapshots, name="load")
n.add("Load", 
      "load_flat", 
      bus="electricity", 
      carrier="AC",
      p_set=load_fix,
      overwrite=True,
      )

n.loads_t.p_set

Load                        load_flat
period timestep                      
2025   2025-01-01 00:00:00       40.0
       2025-01-01 00:30:00       40.0
       2025-01-01 01:00:00       40.0
       2025-01-01 01:30:00       40.0
       2025-01-01 02:00:00       40.0
...                               ...
2049   2049-12-31 21:30:00       40.0
       2049-12-31 22:00:00       40.0
       2049-12-31 22:30:00       40.0
       2049-12-31 23:00:00       40.0
       2049-12-31 23:30:00       40.0

[438000 rows x 1 columns]

In [ ]:
life = end_year-start_year

gas_capex = get_generator_capex(build_costs, tech_params, r, 2025, 'OCGT_SML', 'N3', annuitise=True, lifetime=life)
gas_opex = get_generator_marginal_cost_series(n, tech_params, fuel_costs, 'OCGT_SML', 'GAS', 'N3')

solar_capex = get_generator_capex(build_costs, tech_params, r, 2025, 'SOLAR_PV', 'N3', annuitise=True, lifetime=life)
wind_capex = get_generator_capex(build_costs, tech_params, r, 2025, 'WIND', 'N3', annuitise=True, lifetime=life)

bess_1hr_capex = get_generator_capex(build_costs, tech_params, r, 2025, 'BESS_1HR', 'N3', annuitise=True, lifetime=life)
bess_2hr_capex = get_generator_capex(build_costs, tech_params, r, 2025, 'BESS_2HR', 'N3', annuitise=True, lifetime=life)
bess_4hr_capex = get_generator_capex(build_costs, tech_params, r, 2025, 'BESS_4HR', 'N3', annuitise=True, lifetime=life)
bess_8hr_capex = get_generator_capex(build_costs, tech_params, r, 2025, 'BESS_8HR', 'N3', annuitise=True, lifetime=life)

In [ ]:
modified_gas_opex = apply_fuel_price_volatility(
                        fuel_cost_series=gas_opex,
                        periods_to_modify=[2035, 2036, 2037],
                        min_increase_factor=1.5,
                        max_increase_factor=2,
                        volatility_type="uniform",
                        resolution="annually",  # or "monthly", "quarterly", "annually"
                        random_seed=55  # for reproducible results
                    )

In [ ]:
# Multiple series comparison
fig2 = plot_multiperiod_cost_series(
    cost_series={
        'Base Gas OPEX': gas_opex,
        'Modified Gas OPEX': modified_gas_opex
    },
    title="Gas OPEX Comparison: Base vs Modified",
    y_label="Gas OPEX ($/MWh)",
    colors=['#1f77b4', '#ff7f0e']  # Custom colors if desired
)
fig2.show()

In [ ]:
add_dispatchable_generators(
    n,
    name="OCGT",
    carrier="GAS",
    bus="electricity",
    capital_cost=gas_capex,
    marginal_cost=159,#gas_opex, 
    lifetime=life,
    p_nom_extendable=True,
    build_years=investment_years,
    overwrite=True,
)

In [ ]:
solar_generators = add_vre_generators(
    n,
    name="SOLAR_PV",
    carrier="SOLAR_PV",
    bus="electricity",
    p_max_pu=homer_trace_hourly.SOLAR_TRACE,#max_output_trace.SOLAR_2025,
    capital_cost=solar_capex,
    build_years=investment_years,
    lifetime=life,
    p_nom_extendable=True,
    overwrite=True,
)

wind_generators = add_vre_generators(
    n,
    name="WIND",
    carrier="WIND",
    bus="electricity",
    p_max_pu=homer_trace_hourly.WIND_TRACE,#max_output_trace.WIND_2025,
    capital_cost=wind_capex,
    build_years=investment_years,
    lifetime=life,
    p_nom_extendable=True,
    overwrite=True,
)

In [ ]:
n.generators

In [ ]:
BESS_degradation_series = pd.Series(
    [1.00, 0.95, 0.92, 0.90, 0.88, 0.86, 0.85, 0.83, 0.81, 0.79, 
     0.78, 0.76, 0.74, 0.73, 0.71, 0.69, 0.67, 0.66, 0.64, 0.62, 0.61, 0.59, 0.58, 0.56, 0.55],
    index=range(25)  # Years 0-20 since build
)

BESS_2025 = calc_custom_degradation(
    network_snapshots=n.snapshots,
    technology='BESS',
    build_year=start_year,
    annual_degradation=BESS_degradation_series,
    lifetime=life
)

In [ ]:
# Define storage configurations
storage_configs = {
    # 'BESS_1HR': {
    #     'max_hours': 1,
    #     'capital_cost': bess_1hr_capex,
    #     'carrier': 'BESS'
    # },
    # 'BESS_2HR': {
    #     'max_hours': 2,
    #     'capital_cost': bess_2hr_capex,
    #     'carrier': 'BESS'
    # },
    'BESS_4HR': {
        'max_hours': 4,
        'capital_cost': bess_4hr_capex,
        'carrier': 'BESS'
    },
    # 'BESS_8HR': {
    #     'max_hours': 8,
    #     'capital_cost': bess_8hr_capex,
    #     'carrier': 'BESS'
    # },
    # 'BESS_12HR': {
    #     'max_hours': 12,
    #     'capital_cost': inputs_df.loc[2025, 'BESS_12HR_CAPEX'],
    #     'carrier': 'BESS'
    # },
    # 'BESS_24HR': {
    #     'max_hours': 24,
    #     'capital_cost': inputs_df.loc[2025, 'BESS_24HR_CAPEX'],
    #     'carrier': 'BESS'
    # }
}

# Add all storage units at once
added_storage = add_multiple_storage_units(
    n=n,
    storage_configs=storage_configs,
    #p_max_pu=BESS_2025,
    efficiency_store=0.95,
    efficiency_dispatch=0.95,
    lifetime=life,
    build_years=investment_years,
    p_nom_extendable=True,
    overwrite=True
)

In [ ]:
n.storage_units

In [ ]:
add_link_and_store_bess(
    n,
    name="Battery",
    store_capex_per_mwh=bess_1hr_capex,
    min_dod=0.1,
    max_dod=1,
    bus="electricity",
    carrier="BESS",
)

In [ ]:
n.stores

In [ ]:
n.optimize(solver_name='gurobi', multi_investment_periods=True)

In [ ]:
n.export_to_netcdf("PYPSA_GAS_ONLY_25YRS.nc")

# Results Analysis

In [ ]:
n.import_from_netcdf(r'') # Path to the NetCDF file

In [ ]:
n.snapshot_weightings

In [ ]:
n.snapshot_weightings.generators @ n.generators_t.p.div(1e3)  # GWh

In [ ]:
emissions = (
    n.generators_t.p
    / n.generators.efficiency
    * n.generators.carrier.map(n.carriers.co2_emissions)
)
total_emissions = n.snapshot_weightings.generators @ emissions.sum(axis=1).div(1e6)  # Mt 

In [ ]:
emissions

In [ ]:
import importlib

import src.utils_multiperiod as utils_multiperiod
importlib.reload(utils_multiperiod)

In [ ]:
df = utils_multiperiod.generate_multiperiod_overview(n, renewable_carriers=['SOLAR_PV', 'WIND'], thermal_carriers=['GAS'])
df

In [ ]:
df.to_csv(os.path.join(base_path, 'data', 'GAS_ONLY_PYPSA.csv'))

In [ ]:
total_cost = n.objective / 1e6 # $Million
print(f"Annual system costs: ${np.round(total_cost,2)}M")

In [ ]:
plot_monthly_electric_production(n, start_year="2025", end_year="2045")

In [ ]:
plot_generator_output_heatmap(n, start_date="2025", end_date="2030", carrier="GAS").show()


In [ ]:
plot_storage_soc_heatmap(n, start_date="2026", end_date="2045").show()

In [ ]:
plot_storage_soc(n, start_date="2028", end_date="2029")

In [ ]:
create_dispatch_plot(n, start_date="2034", end_date="2035", stack=True, interactive=True, y_range=[-20, 60])

In [ ]:
create_dispatch_plot_with_curtailment(n, start_date="2034", end_date="2035", stack=True, y_range=[-20, 60], vre_carriers=["SOLAR_PV", "WIND"])

In [ ]:
n.model

# Sensitivity Analysis

In [ ]:
# Example of how to calculate sensitivity analysis for CO2 limits

# sensitivity = {}
# for co2 in [150, 100, 50, 25, 0]:
#     n.global_constraints.loc["CO2Limit", "constant"] = co2 * 1e6
#     n.optimize(solver_name="gurobi")
#     sensitivity[co2] = system_cost(n)